## Contextual Compression 
 ### Instead of immeditely returing the retrieved documents, we will compress and return to llm

In [44]:
from langchain_ollama.chat_models import ChatOllama

In [45]:
model_name = "llama3.2:1b"

In [46]:
llm = ChatOllama(model=model_name)

In [48]:
query = "What did the president say about Ketanji Brown Jackson?"

In [49]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
embedding = OllamaEmbeddings(model=model_name)

documents = TextLoader("/Users/mohamednajiaboo/Desktop/Langchain/samples/state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts,embedding=embedding ).as_retriever()

docs = retriever.invoke(query)
docs

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


[Document(metadata={'source': '/Users/mohamednajiaboo/Desktop/Langchain/samples/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': '/Users/mohamednajiaboo/Desktop/Langchain/samples/sta

In [50]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [55]:
from langchain_openai import OpenAI

In [56]:
llm_openai = OpenAI(temperature=0)

In [57]:
compressor = LLMChainExtractor.from_llm(llm=llm_openai)

In [58]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [59]:
compressed_docs = compression_retriever.invoke(query)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


In [61]:
compressed_docs[0].page_content

'I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson.'

## Flash Reranker
 ### Rerank the result 

In [62]:
from langchain.retrievers.document_compressors import FlashrankRerank

In [64]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever, base_compressor=compressor
)

In [65]:
rerank_document = compression_retriever.invoke(query)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [66]:
rerank_document

[Document(metadata={'id': 0, 'relevance_score': 0.9990259, 'source': '/Users/mohamednajiaboo/Desktop/Langchain/samples/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'id': 3, 'relevance_scor

In [67]:
retriever.invoke(query)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


[Document(metadata={'source': '/Users/mohamednajiaboo/Desktop/Langchain/samples/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': '/Users/mohamednajiaboo/Desktop/Langchain/samples/sta